# Lightricks Challenge - DataHack 2018

In [ ]:
import os
import sys
import operator
import numpy as np
import pandas as pd
import scipy
import dateutil
from sklearn.model_selection import train_test_split,KFold,cross_val_score
# from sklearn.cross_validation import train_test_split, KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn import tree
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from sklearn import svm
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set()

In [ ]:
path = './'
teamname = 'MadeInJerusalem'
out_name = path + teamname + '_submission.csv'

In [ ]:
df_usage = pd.read_csv(path + "train_usage_data.csv")

In [ ]:
#Looking data format and types
df_usage.info()

The variables in dataframe "df_usage" are:

| Field name   | Description | 
|----------|-------------|
| id | User ID |
| feature_name | Name of feature used |
| usage_duration | Duration in seconds between feature was entered and accepted or canceled |
| use_date | Date and time when the feature was entered |
| accepted | True if the user accepted the changes by the feature and False if he did not accepted the changes |

In [ ]:
df_usage.head()

In [ ]:
df_users = pd.read_csv(path + "train_users_data.csv",parse_dates = [ 'installation_date','subscripiton_date'])

In [ ]:
df_users.info()

In [ ]:
df_users.head()

The variables in dataframe "df_users" are:

| Field name   | Description | 
|----------|-------------|
| id | User ID |
| installation_date | Date and time when the application was first installed |
| subscription_date | Date and time when the user joined as a subscriber |
| country | Country where the user is based |
| days_until_churned | Days before the user decided to leave the subscription (churn). NaN if the user has not churned |
| churned | False if the user is still a subscriber and True if the user stopped his subscription. |

In [ ]:
df_users = df_users.drop(columns='Unnamed: 0')
df_users.head()

# Data exploration:

In [ ]:
df_users['churned'].value_counts().plot('bar')

In [ ]:
joined_df = df_usage.join(df_users.set_index('id'),on='id')

In [ ]:
joined_df.head()

In [ ]:
total_number_of_usage_per_feature = df_usage['feature_name'].value_counts()
total_number_of_usage_per_feature.shape

In [ ]:
k = 10
top_k_features_churned = joined_df[joined_df['churned']==True]['feature_name'].value_counts().index[:k]
top_k_features_not_churned = joined_df[joined_df['churned']==False]['feature_name'].value_counts().index[:k]

In [ ]:
top_k_features_not_churned

In [ ]:
df_churned = joined_df[(joined_df['churned']==True) & (joined_df['accepted']==True)]
df_churned.head()

In [ ]:
#What are the most popular features, within each class:

#total_number_of_usage_per_feature = df_usage['feature_name'].value_counts()



# k = 10
# top_k_features_churned = joined_df[joined_df['churned']==True]['feature_name'].value_counts().index[:k]
# top_k_features_not_churned = joined_df[joined_df['churned']==False]['feature_name'].value_counts().index[:k]


fig, ax = plt.subplots(2,1, figsize=(17,12))
df_churned = joined_df[(joined_df['churned']==True) & (joined_df['accepted']==True)]
p = sns.countplot(data=df_churned[df_churned['feature_name'].isin(top_k_features_churned)], x='feature_name', order = top_k_features_churned, ax=ax[0])

df_not_churned = joined_df[(joined_df['churned']==False) & (joined_df['accepted']==True)]
q = sns.countplot(data=df_not_churned[df_not_churned['feature_name'].isin(top_k_features_not_churned)], x='feature_name', order = top_k_features_not_churned, ax=ax[1])

ax[0].set_title('Most used features, churned=1')
ax[1].set_title('Most used features, churned=0')

In [ ]:
groupedDf = joined_df.groupby(['id', 'churned', 'feature_name'])
groupedDf.mean().head()

In [ ]:
joined_df.groupby(['id', 'churned'])[['accepted']].mean().boxplot(by='churned')

# TODO: Filter outliers

In [ ]:
#end_of_time_series = np.log(joined_df['usage_duration']).hist()
joined_df = joined_df[joined_df['usage_duration']<200]

# Creating our input data:

In [ ]:
# Add time statistics
start_of_time_series = joined_df['subscripiton_date'].min()
print(start_of_time_series)
end_of_time_series = joined_df['use_date'].max()
print(end_of_time_series)
date_features = joined_df[['id','installation_date','subscripiton_date','use_date']].groupby('id').max()
date_features['days_installed'] = (pd.to_datetime(end_of_time_series) - date_features['installation_date']).dt.days
date_features['days_installed_not_subscribed'] = (date_features['subscripiton_date'] - date_features['installation_date']).dt.days
date_features['days_since_last_use'] = (pd.to_datetime(end_of_time_series) - pd.to_datetime(date_features['use_date'])).dt.days
date_features['days_used'] = np.minimum((pd.to_datetime(date_features['use_date']) - date_features['installation_date']).dt.days, 
                                        (pd.to_datetime(date_features['use_date']) - pd.to_datetime(start_of_time_series) ).dt.days)
date_features = date_features[['days_installed','days_installed_not_subscribed','days_since_last_use','days_used']]
date_features.head()

In [ ]:
# extract mean usage time for FaceTune features
users_mean_usage_time = pd.pivot_table(joined_df[['id', 'feature_name', 'usage_duration']], values='usage_duration', index=['id'], columns=['feature_name'], aggfunc=np.mean, fill_value=0)
users_mean_usage_time = users_mean_usage_time.add_suffix('_mean_time')
users_mean_usage_time.head()

In [ ]:
# extract mean acceptance rate for FaceTune features
users_mean_acceptance_rate = pd.pivot_table(joined_df[['id', 'feature_name', 'accepted']], values='accepted', index=['id'], columns=['feature_name'], aggfunc=np.mean, fill_value=0)
users_mean_acceptance_rate = users_mean_acceptance_rate.add_suffix('_mean_acceptance')
users_mean_acceptance_rate.head()

In [ ]:
#Let's create a table with statistic summaries: rows correspond to users; columns to various statistics:
users_usage_summaries = pd.pivot_table(joined_df[['id', 'feature_name']], index=['id'], columns=['feature_name'], aggfunc=len, fill_value=0)
users_usage_summaries.head()

In [ ]:
#Let's add the mean of 'accepted' for each user:
accepted_rate = joined_df.groupby(['id'])['accepted'].mean().to_frame()
accepted_rate.head()

In [ ]:
# Add labels
churned = joined_df.groupby(['id'])['churned'].mean().to_frame()
churned.head()

In [ ]:
#Assemble all the features
#total_features = users_usage_summaries.join(accepted_rate, how='left').join(churned, how='left')
total_features = users_usage_summaries.join(users_mean_usage_time, how='left').join(users_mean_acceptance_rate, how='left').join(date_features, how='left').join(accepted_rate, how='left').join(churned, how='left')

#This is how our df looks like:
total_features.shape


In [ ]:
#number_of_churned = total_features[total_features['churned']==True].shape[0]
#churned_samples = total_features[total_features['churned']==True]
#not_churned_samples = total_features[total_features['churned']==False].sample(n=number_of_churned)
#total_features = pd.concat([churned_samples, not_churned_samples]).sample(frac=1)
#This is how our df looks like:
#total_features.shape

# Learn a decision tree:

In [ ]:
algo = make_pipeline(preprocessing.MinMaxScaler(), svm.LinearSVC(class_weight='balanced'))
scores = cross_val_score(algo, X, y, cv=5, scoring='f1')
scores.mean()
print(np.round(scores.mean(),3))

In [ ]:
X = total_features.iloc[:, total_features.columns!='churned'].values
y = total_features.loc[:,'churned'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

In [ ]:
#algo=tree.DecisionTreeClassifier(criterion='entropy',max_depth = 8)
scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
algo  = svm.LinearSVC(class_weight='balanced')
train = algo.fit(X_train_scaled, y_train)
X_val_scaled = scaler.transform(X_val)
res_train=train.predict(X_train_scaled)
res=train.predict(X_val_scaled)
print(res)

In [ ]:
#Let's check the confusion matrix:
cm = metrics.confusion_matrix(y_train, res_train)
print(cm)
print(classification_report(y_pred=res_train,y_true=y_train))
# print only f1 score for positive
print(np.round(f1_score(y_pred=res_train,y_true=y_train),3))

In [ ]:
#Let's check the confusion matrix:
cm = metrics.confusion_matrix(y_val, res)
print(cm)
print(classification_report(y_pred=res,y_true=y_val))
# print only f1 score for positive
print(np.round(f1_score(y_pred=res,y_true=y_val),3))

In [ ]:
#test your predictor:

#1.Prepare your test-set (in case you created new features/transformed the input data):
df_usage_test = pd.read_csv("test_usage_data.csv")
users_usage_summaries_test = pd.pivot_table(df_usage_test[['id', 'feature_name']], index=['id'], columns=['feature_name'], aggfunc=len, fill_value=0)

accepted_rate_test = df_usage_test.groupby(['id'])['accepted'].mean().to_frame()

#Let's merge the two:
users_usage_summaries_test = users_usage_summaries_test.join(accepted_rate_test, how='left')
X_test = users_usage_summaries_test.values

In [ ]:
users_usage_summaries

In [ ]:
#submit result:
pred = train.predict(X_test)
df = pd.DataFrame(pred, index=users_usage_summaries_test.index.astype(str), columns=['churned'], dtype=str)
df.to_csv(out_name, header=True, quoting=csv.QUOTE_NONNUMERIC) 